In [1]:
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
src = pathlib.Path().cwd().parent.parent / 'data' / 'processed'
ref = pathlib.Path().cwd().parent.parent / 'reference'

In [3]:
ct = pd.read_csv(src / 'ct-from-test-data' / 'ct.csv', index_col='date_sample')
ct_by_awh = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-awh.csv')
ct_by_adh = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-adh.csv')
ct_by_sex = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-sex.csv')
ct_by_ag = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-ag.csv')
addiv = pd.read_csv(ref / 'addiv.csv')[['id_addiv', 'name_addiv_2']]
pop = pd.read_csv(ref / 'pop_1.csv')[['id_addiv', 'pop']]

In [4]:
ct

,no_test,no_positive,no_positive_group_sample,no_test_corrected,pct_positive,no_test_rollsum,no_positive_rollsum,pct_positive_per7d,ct
date_sample,,,,,,,,,
2020-07-04,2.0,0.0,0.0,2.0,0.000000,NaN,NaN,0.000000,1
2020-07-05,46.0,4.0,0.0,46.0,0.086957,NaN,NaN,0.000000,1
2020-07-06,1.0,0.0,0.0,1.0,0.000000,NaN,NaN,0.000000,1
2020-07-07,281.0,21.0,0.0,281.0,0.074733,NaN,NaN,0.000000,1
2020-07-08,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,0.000000,1
...,...,...,...,...,...,...,...,...,...
2021-09-22,24086.0,5029.0,1383.0,22703.0,0.221513,399467.0,49058.0,0.122809,3
2021-09-23,17470.0,4550.0,332.0,17138.0,0.265492,401633.0,48648.0,0.121126,3
2021-09-24,13389.0,4228.0,403.0,12986.0,0.325581,364799.0,46083.0,0.126324,3


In [5]:
tphcm = {
    'addr_dist_home':['THANH PHO HO CHI MINH'],
    'no_test_corrected': [ct[['no_test_corrected', 'no_positive']].sum()[0]],
    'no_positive': [ct[['no_test_corrected', 'no_positive']].sum()[1]]
}

df_tphcm = pd.DataFrame(tphcm)

df = (
    ct_by_adh[['date_sample', 'addr_dist_home', 'no_test_corrected', 'no_positive']]
#     .query('date_sample == "2021-08-05"')
    .groupby('addr_dist_home')[['no_test_corrected', 'no_positive']]
    .sum()
#     .to_frame(name='no_test_cumsum')
    .reset_index()
    .append(df_tphcm)
    .sort_values('no_test_corrected')
    .merge(
        addiv,
        how = 'left',
        left_on = 'addr_dist_home',
        right_on = 'name_addiv_2'
    )
    .merge(
        pop,
        how = 'left',
        left_on = 'id_addiv',
        right_on = 'id_addiv'
    )
    .drop(columns={'id_addiv', 'name_addiv_2'})
)

df['pct_positive'] = df['no_positive'] / df['no_test_corrected']

df['predicted_inc'] = df['pop'] * df['pct_positive']

df.to_csv(src / 'predicted-inc-from-test-data' / 'predicted-inc.csv', index=False)
df

,addr_dist_home,no_test_corrected,no_positive,pop,pct_positive,predicted_inc
0,HUYEN CAN GIO,24599.0,2340.0,71526,0.095126,6.803969e+03
1,QUAN 11,28145.0,8405.0,209867,0.298632,6.267302e+04
2,QUAN PHU NHUAN,34358.0,6962.0,163961,0.202631,3.322360e+04
3,QUAN 06,38829.0,18505.0,233561,0.476577,1.113098e+05
4,QUAN 01,39233.0,14840.0,142625,0.378253,5.394833e+04
5,HUYEN NHA BE,40375.0,21000.0,206837,0.520124,1.075809e+05
6,QUAN 05,42337.0,15017.0,159073,0.354702,5.642344e+04
7,QUAN 02,44082.0,4657.0,180275,0.105644,1.904498e+04
8,QUAN 04,48676.0,18709.0,175329,0.384358,6.738907e+04
9,QUAN TAN PHU,65554.0,22592.0,485348,0.344632,1.672664e+05


In [6]:
# df = (
#     ct_by_adh[['addr_dist_home', 'date_sample', 'no_test', 'no_positive']]
#     .groupby(['addr_dist_home', 'date_sample']).sum()
#     .groupby(level=0).cumsum().reset_index()
#     .merge(
#         addiv,
#         how = 'left',
#         left_on = 'addr_dist_home',
#         right_on = 'name_addiv_2'
#     )
#     .merge(
#         pop,
#         how = 'left',
#         left_on = 'id_addiv',
#         right_on = 'id_addiv'
#     )
#     .drop(columns={'id_addiv', 'name_addiv_2'})
# )

# df['pct_positive'] = df['no_positive'] / df['no_test']

# df['predicted_inc'] = df['pop'] * df['pct_positive']
# # foo[foo.addr_dist_home == 'HUYEN BINH CHANH'].tail(10)
# df